In [30]:
from abc import ABC, abstractmethod
import json
from typing import List, NamedTuple
from openai import OpenAI
from openai.types.chat import ChatCompletion

import re
import os
from clef.utils.data_loading import AuthorityPost

import logging

from clef.verification.models.open_ai import BaseVerifier
logger = logging.getLogger(__name__)

class VerificationResult(NamedTuple):
    label: str
    score: float
    
class OpenaiVerifier(BaseVerifier):
    client: OpenAI
    model: str = "gpt-4-turbo-preview"
    valid_labels: List =  [
            "REFUTES",
            "NOT ENOUGH INFO",
            "SUPPORTS"
        ]

    def __init__(self, api_key:str='') -> None:
        self.client = OpenAI(
            api_key=(api_key or os.environ.get("OPENAI_API_KEY")),
        )
        self.assistant_id: str = "asst_XRITdOybDfYpIr4fVevm6qYi"
        self.total_tokens_used: int = 0
    
    def get_assistant_response(self, input_message):
        thread = self.client.beta.threads.create()
        message = self.client.beta.threads.messages.create(
            thread_id=thread.id,
            role="user",
            content=input_message
        )

        run = self.client.beta.threads.runs.create_and_poll(
            thread_id=thread.id,
            assistant_id=self.assistant_id,
        )

        if run.status == 'completed':
            self.total_tokens_used += run.usage.total_tokens
            messages = self.client.beta.threads.messages.list(
                thread_id=thread.id,
                limit=1
            )
            if messages:
                for message in messages.data:
                    if message.role == "assistant":
                        return messages.data.content[0].text.value # type: ignore
        else:
            logger.warn(f'could not unpack response from openai model: {run}')
            return '{"decision": "NOT ENOUGH INFO", confidence": 1.0}'

    def verify(self, claim: str, evidence: str) -> VerificationResult:
        return VerificationResult('', 1.0)

In [31]:
oaiver = OpenaiVerifier()
oaiver

In [32]:
res = oaiver.get_assistant_response('hello')

In [39]:
res.data[0].content[0].text.value

'{"decision": "NOT ENOUGH INFO", "confidence": 1}'